In [2]:
# code source: https://arkalsekar.medium.com/how-to-get-all-the-co-ordinates-of-hand-using-mediapipe-hand-solutions-ac7e2742f702
import cv2

import glob
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image as PILImage
import cv2 as cv
from IPython.display import display, Image, clear_output

from io import StringIO

import mediapipe as mp
import ipywidgets as widgets

Import Image

In [3]:
image_src = '../data/fretboard_dataset/fretboard_frames_test/28_0.png'
image = cv2.imread(image_src)
h,w,c = image.shape
out = widgets.Output(layout={'border': '1px solid black', 'width': f'{w/2}px', 'height': f'{h/2}px'})
index = 0
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

with out:
    with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)
        image_height, image_width, _ = image.shape
        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            file_path_coords = f"image_{index}.txt"
            file = open(file_path_coords, 'a')
            for hand_landmarks in results.multi_hand_landmarks:
                # Here is How to Get All the Coordinates
                for ids, landmrk in enumerate(hand_landmarks.landmark):
                    # print(ids, landmrk)
                    # cx, cy = landmrk.x * image_width, landmrk.y*image_height
                    # print(cx, cy)'
                    coord = [ids, landmrk.x, landmrk.y]
                    file.write(str(coord))
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            file.close()
            
        file_path_img = f"image_{index}.png"
        cv2.imwrite(file_path_img, image)
        
        index += 1
        display(Image(data=cv2.imencode('.png', image)[1].tobytes()))
        
display(out)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Image Capture

In [9]:
camera_source = 0

# Open the camera
cap = cv2.VideoCapture(camera_source)
if not cap.isOpened():
    print("Error: Couldn't open the camera.")

out = widgets.Output(layout={'border': '1px solid black', 'width': '400px', 'height': '300px'})
index = 0
def on_button_clicked(_):
    global index
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands

    with out:
        with mp_hands.Hands(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
            clear_output(wait=True)
            ret, image = cap.read()
            
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            results = hands.process(image)
            image_height, image_width, _ = image.shape
            # Draw the hand annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:
                file_path_coords = f"image_{index}.txt"
                file = open(file_path_coords, 'a')
                for hand_landmarks in results.multi_hand_landmarks:
                    # Here is How to Get All the Coordinates
                    for ids, landmrk in enumerate(hand_landmarks.landmark):
                        # print(ids, landmrk)
                        # cx, cy = landmrk.x * image_width, landmrk.y*image_height
                        # print(cx, cy)'
                        coord = [ids, landmrk.x, landmrk.y]
                        file.write(str(coord))
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                file.close()
                
            file_path_img = f"image_{index}.png"
            cv2.imwrite(file_path_img, image)
            
            index += 1
            display(Image(data=cv2.imencode('.png', image)[1].tobytes()))
        
        
button = widgets.Button(description="Capture Frame")
button.on_click(on_button_clicked)
display(button)
display(out)

Button(description='Capture Frame', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [10]:
cap.release()

Video Capture

In [15]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Configure MediaPipe Hands model
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        # Read a frame from the camera
        success, image = cap.read()

        # Check for the end of the video stream
        if not success:
            print("Ignoring empty camera frame.")
            break

        # Preprocess the image
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image with MediaPipe Hands
        results = hands.process(image)

        # Extract image dimensions
        image_height, image_width, _ = image.shape

        # Revert back to BGR for rendering annotations
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw hand landmarks on the image
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for ids, landmrk in enumerate(hand_landmarks.landmark):
                    cx, cy = landmrk.x * image_width, landmrk.y * image_height
                    print(cx, cy)

                # Draw landmarks and connections
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display the annotated image
        cv2.imshow('MediaPipe Hands', image)

        # Break the loop if the 'Esc' key is pressed
        if cv2.waitKey(5) & 0xFF == 27:
            break


In [13]:
cap.release()